<a href="https://colab.research.google.com/github/enzoampil/tito-joker/blob/master/experiments/Jokes_GPT2_Finetuning2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tito Joker v2.1

The hope is to address the missing token issue from v2 #22
## Updates:
1. Set sequence length to 100 (closer to typical joke length)

## Note that the experiment notebooks have ending numbers corresponding to the model's number (this is model2)

In [2]:
!pip install transformers

In [3]:
!git clone https://github.com/huggingface/transformers.git

Cloning into 'transformers'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 23527 (delta 3), reused 4 (delta 0), pack-reused 23509
Receiving objects: 100% (23527/23527), 14.09 MiB | 19.76 MiB/s, done.
Resolving deltas: 100% (16656/16656), done.


In [4]:
!git clone https://github.com/enzoampil/tito-joker.git

Cloning into 'tito-joker'...
remote: Enumerating objects: 125, done.
remote: Counting objects: 100% (125/125), done.
remote: Compressing objects: 100% (56/56), done.
remote: Total 478 (delta 67), reused 113 (delta 59), pack-reused 353
Receiving objects: 100% (478/478), 6.10 MiB | 11.96 MiB/s, done.
Resolving deltas: 100% (263/263), done.


## Download raw jokes dataset

In [5]:
!wget https://storage.googleapis.com/joke-generator-model1/short-jokes.zip && unzip short-jokes.zip

--2020-04-08 13:05:35--  https://storage.googleapis.com/joke-generator-model1/short-jokes.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c03::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10679873 (10M) [application/zip]
Saving to: ‘short-jokes.zip’

short-jokes.zip     100%[===================>]  10.18M  58.6MB/s    in 0.2s    

2020-04-08 13:05:35 (58.6 MB/s) - ‘short-jokes.zip’ saved [10679873/10679873]

Archive:  short-jokes.zip
  inflating: shortjokes.csv          


## Process the raw jokes

In [6]:
%cd tito-joker

/content/tito-joker


## Prepare dataset

In [0]:
!python ./src/utils/process_jokes.py ../shortjokes.csv ./data/riddle_jokes.txt

In [8]:
ls data

riddle_jokes.txt  shortjokes_sample100.csv


In [0]:
#from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer
#from pathlib import Path

In [0]:
from transformers import GPT2Model, GPT2Config

## Save tokenizer w/ new tokens

In [0]:
!mkdir models
!mkdir models/pretrained
!mkdir models/finetuned

In [12]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
new_tokens = ['<soq>', '<eoq>']
num_added_toks = tokenizer.add_tokens(new_tokens)

In [13]:
tokenizer._convert_token_to_id("<|endoftext|>")

50256

In [14]:
tokenizer.save_pretrained('./models/pretrained')

('./models/pretrained/vocab.json',
 './models/pretrained/merges.txt',
 './models/pretrained/special_tokens_map.json',
 './models/pretrained/added_tokens.json')

In [0]:
import json
with open('./models/pretrained/config.json', 'w') as f:
    json.dump(GPT2Config().to_dict(), f)

## Finetune the GPT2 model

In [16]:
!python ../transformers/examples/run_language_modeling.py \
    --overwrite_output_dir \
    --output_dir=./models/finetuned \
    --model_type=gpt2\
    --model_name_or_path=gpt2\
    --tokenizer_name ./models/pretrained \
    --do_train\
    --train_data_file=./data/riddle_jokes.txt \
    --per_gpu_train_batch_size 4 \
    --block_size 100 # Set this arbitrarily for now. But most jokes will be within this range. We don't want to weight super long jokes anyway.\

04/08/2020 13:09:40 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
04/08/2020 13:09:40 - INFO - filelock -   Lock 139800749889744 acquired on /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.699bbd1c449e9861456f359d6daa51bd523ac085b4b531ab0aad5a55d091e942.lock
04/08/2020 13:09:40 - INFO - transformers.file_utils -   https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpounov0u3
Downloading: 100% 224/224 [00:00<00:00, 230kB/s]
04/08/2020 13:09:40 - INFO - transformers.file_utils -   storing https://s3.amazonaws.com/models.huggingface.co/bert/gpt2-config.json in cache at /root/.cache/torch/transformers/4be02c5697d91738003fb1685c9872f284166aa32e061576bbe6aaeb95649fcf.699bbd1c449e9861456f359d6daa51bd523ac085b4b531ab0aad5a55d091e942
04/08/2020 13:09:40 - INFO

## Test results

In [17]:
%cd /content

/content


In [18]:
!python transformers/examples/run_generation.py \
    --model_type=gpt2 \
    --model_name_or_path=tito-joker/models/finetuned/ \
    --length=50 \
    --stop_token='<|endoftext|>' \
    --num_return_sequences 50

04/08/2020 13:17:47 - INFO - transformers.tokenization_utils -   Model name 'tito-joker/models/finetuned/' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'tito-joker/models/finetuned/' is a path, a model identifier, or url to a directory containing tokenizer files.
04/08/2020 13:17:47 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/vocab.json
04/08/2020 13:17:47 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/merges.txt
04/08/2020 13:17:47 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/added_tokens.json
04/08/2020 13:17:47 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/special_tokens_map.json
04/08/2020 13:17:47 - INFO - transformers.tokenization_utils -   loading file tito-joker/models/finetuned/tokenizer_config.json
04/08/2020 13:17:47 - INFO - transformers.configuration_utils - 

## Save results to gcs

In [22]:
%cd tito-joker/models/finetuned

/content/tito-joker/models/finetuned


In [23]:
ls

added_tokens.json  checkpoint-3000/  pytorch_model.bin
checkpoint-1000/   checkpoint-3500/  special_tokens_map.json
checkpoint-1500/   checkpoint-500/   tokenizer_config.json
checkpoint-2000/   config.json       training_args.bin
checkpoint-2500/   merges.txt        vocab.json


In [0]:
mkdir model2-1

In [0]:
mv added_tokens.json config.json merges.txt pytorch_model.bin special_tokens_map.json tokenizer_config.json training_args.bin vocab.json model2-1

In [43]:
!zip -r model2-1.zip  model2-1

  adding: model2-1/ (stored 0%)
  adding: model2-1/vocab.json (deflated 63%)
  adding: model2-1/training_args.bin (deflated 37%)
  adding: model2-1/added_tokens.json (deflated 22%)
  adding: model2-1/pytorch_model.bin (deflated 16%)
  adding: model2-1/special_tokens_map.json (deflated 52%)
  adding: model2-1/merges.txt (deflated 53%)
  adding: model2-1/tokenizer_config.json (deflated 48%)
  adding: model2-1/config.json (deflated 57%)


In [45]:
ls -l model2-1

total 536636
-rw-r--r-- 1 root root        32 Apr  8 13:17 added_tokens.json
-rw-r--r-- 1 root root      1332 Apr  8 13:17 config.json
-rw-r--r-- 1 root root    456318 Apr  8 13:17 merges.txt
-rw-r--r-- 1 root root 548130025 Apr  8 13:17 pytorch_model.bin
-rw-r--r-- 1 root root        90 Apr  8 13:17 special_tokens_map.json
-rw-r--r-- 1 root root       107 Apr  8 13:17 tokenizer_config.json
-rw-r--r-- 1 root root      1263 Apr  8 13:17 training_args.bin
-rw-r--r-- 1 root root    898669 Apr  8 13:17 vocab.json


In [0]:
from google.colab import auth
auth.authenticate_user()

In [47]:
!gsutil cp model2-1.zip gs://joke-generator-model1/

Copying file://model2-1.zip [Content-Type=application/zip]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

/
Operation completed over 1 objects/441.6 MiB.                                    
